In [5]:
import json
from pathlib import Path
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [6]:
data = json.loads(Path("../data/tlds_data_mc4_tw_000.nlp.json").read_text(encoding="UTF-8"))
train_data, test_data = train_test_split(
                          data, test_size=0.2, 
                          stratify=[x["tld"] for x in data],
                          random_state=12345)

In [8]:
Counter([x["tld"] for x in train_data])

Counter({'com': 200, 'org': 200, 'gov': 200, 'edu': 200})

In [50]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def to_words(item_x):
  return [x[0] for x in item_x["nlp"]]
tfidf_trans = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False, min_df=1)
pipe = Pipeline([
      ('scaler', StandardScaler(with_mean=False)), 
      ('logistic', LogisticRegression(random_state=12345))])
X = tfidf_trans.fit_transform([to_words(x) for x in train_data])
print(X.shape)
y = [x["tld"] for x in train_data]
pipe.fit(X,y)
pipe.score(X,y)

(800, 28586)


1.0

In [51]:
testX = tfidf_trans.transform([to_words(x) for x in test_data])
testy = [x["tld"] for x in test_data]
pipe.score(testX,testy)

0.71

In [52]:
import numpy as np
feats = tfidf_trans.get_feature_names_out()
clf = pipe["logistic"]
for i in range(4):
  tld_feat_rank = np.argsort(-clf.coef_[i])
  print(clf.classes_[i], [feats[x] for x in tld_feat_rank[:10]])

com ['購物', '新浪', '情報', '上午點', '商品', '團號', '出發', '天夜', '字級', '電視']
edu ['取自', '修訂', '花絮', '夏季班', '研究', '貢獻', '對話', '大學', '參訪', '做']
gov ['人次', '瀏覽', '好客', '標籤', '影音', '客家', '平台', '集', '總瀏覽', '政府']
org ['推文', '使者', '雜誌', '台灣', '協會', '專欄', '建議', '能源局', '包膜', '鋼化']


In [55]:
clf.coef_.size

114344

In [58]:
text = ("臺灣 大學 生物 資源 暨 農學院 在 區塊鏈 應用 於 智慧 農業 的 技術 上 ， 數年 前 即 「 超前 部署 」 ， "
       "如今 已 進入 「 數位 農業 創新 知識 履歷 碳 溯源 」 階段 ， 成果 豐碩 ， 預期 知識 區塊鏈 將 會 翻轉 臺灣 農業 的 新 面貌 。")
clf.predict(tfidf_trans.transform([text.split()]))

array(['com'], dtype='<U3')

In [68]:
## OOVs:
[x for x in text.split() if x not in feats]

['數年', '「', '」', '「', '」']

## With Transformer

In [60]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("../data/bonjour-tlds")
model = AutoModelForSequenceClassification.from_pretrained("../data/bonjour-tlds")

In [66]:
import torch
with torch.no_grad():
  logits = model(**tokenizer(text.replace(" ", ""), return_tensors="pt")).logits

pred = np.argmax(logits, axis=-1)
["com", "edu", "gov", "org"][pred]

'edu'